In [1]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import exp

np.set_printoptions(suppress=True)

%load_ext lab_black
%load_ext watermark

# Gastric cancer example

Adapted from code for [Unit 8: gastric.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit8/gastric.odc).

Data can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/gastric.txt).

Associated lecture video: Unit 8 lesson 6

## Problem statement
Stablein et al. (1981) provide data on 90 patients affected by locally advanced, nonresectable gastric carcinoma. The patients are randomized to two treatments: chemotherapy alone (coded as 0) and chemotherapy plus radiation (coded as 1). Survival time is reported in days. Recorded times are censored if the patient stopped participating in the study before it finished.

Stablein, D. M., Carter, W. H., Novak, J. W. (1981). Analysis of survival data with nonproportional hazard functions. Control. Clin. Trials,  2 , 2, 149--159.

### Data
Columns are, from left to right:
- type: Treatment type, chemotherapy (0) or chemotherapy + radiation (1)
- censored: If censored, meaning the patient survived the observation period, the time in days appears here rather than in the times column. 0 if not censored.
- times: Recorded days without cancer recurrence. NaN if censored.

### Model changes
PyMC really did not like the noninformative exponential prior on v (α in this model). It's a very broad prior. To avoid divide by zero errors, I just kept increasing lambda until the model ran all the way through. This is not ideal, but I haven't had time to look into it further. The results actually came out fairly close to the BUGS results.

## Method 1: ```pm.Censored```

The way PyMC censoring works is described in some detail in [this notebook](https://docs.pymc.io/projects/examples/en/latest/generalized_linear_models/GLM-truncated-censored-regression.html#censored-regression-model) by [Dr. Benjamin T. Vincent](https://github.com/drbenvincent). This is accomplished in the source code [here](https://github.com/aesara-devs/aeppl/blob/751979802f1aef5478fdbf7cc1839df07df60825/aeppl/truncation.py#L79) if you want to take a look. For right-censoring, try this: ```pm.Censored("name", dist, lower=None, upper=censored, observed=y)```. The censored values can be an array of the same shape as the y values. 

If the y value equals the right-censored value, [```pm.Censored```](https://docs.pymc.io/en/latest/api/distributions/generated/pymc.Censored.html#pymc.Censored) returns the complement to the CDF evaluated at the censored value. If the y value is greater than the censored value, it returns ```-np.inf```. Otherwise, the distribution you passed to the ```dist``` parameter works as normal. What I've been doing is setting the values in the censored array to ```np.inf``` if the corresponding y value is not censored, and equal to the y value if it should be censored.

```{note}
I've noticed that this method is unstable with some distributions. Try using the imputed censoring model (below) if this one isn't working.
```

In [2]:
data = np.loadtxt("../data/gastric.txt")
data.shape

(90, 3)

In [3]:
x = data[:, 0].copy()
censored = data[:, 1].copy()
y = data[:, 2].copy()
# for pymc, right-censored values must be greater than or equal to than the "upper" value
y[np.isnan(y)] = censored[np.isnan(y)]
censored[censored == 0] = np.inf

In [4]:
y

array([  17.,   42.,   44.,   48.,   60.,   72.,   74.,   95.,  103.,
        108.,  122.,  144.,  167.,  170.,  183.,  185.,  193.,  195.,
        197.,  208.,  234.,  235.,  254.,  307.,  315.,  401.,  445.,
        464.,  484.,  528.,  542.,  567.,  577.,  580.,  795.,  855.,
        882.,  892., 1031., 1033., 1306., 1335., 1366., 1452., 1472.,
          1.,   63.,  105.,  129.,  182.,  216.,  250.,  262.,  301.,
        301.,  342.,  354.,  356.,  358.,  380.,  381.,  383.,  383.,
        388.,  394.,  408.,  460.,  489.,  499.,  524.,  529.,  535.,
        562.,  675.,  676.,  748.,  748.,  778.,  786.,  797.,  945.,
        955.,  968., 1180., 1245., 1271., 1277., 1397., 1512., 1519.])

In [5]:
censored

array([  inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
        882.,  892., 1031., 1033., 1306., 1335.,   inf, 1452., 1472.,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,  381.,   inf,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,  529.,   inf,
         inf,   inf,   inf,   inf,   inf,   inf,   inf,   inf,  945.,
         inf,   inf, 1180.,   inf,   inf, 1277., 1397., 1512., 1519.])

```{warning}
PyMC and BUGS do not specify the Weibull distribution in the same way!

α = v

β = λ ** (-1 / α)

```

In [12]:
log2 = np.log(2)

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.01)
    beta1 = pm.Normal("beta1", 0, tau=0.1)
    α = pm.Exponential("α", 4)

    λ = exp(beta0 + beta1 * x)
    β = λ ** (-1 / α)

    obs_latent = pm.Weibull.dist(alpha=α, beta=β)
    likelihood = pm.Censored(
        "likelihood",
        obs_latent,
        lower=None,
        upper=censored,
        observed=y,
    )

    median0 = pm.Deterministic("median0", (log2 * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (log2 * exp(-beta0 - beta1)) ** (1 / α))

    # update these variable names later
    S = pm.Deterministic("S", exp(-λ * (likelihood**α)))
    f = pm.Deterministic("f", λ * α * (likelihood ** (α - 1)) * S)
    h = pm.Deterministic("h", f / S)

    trace = pm.sample(
        10000,
        tune=2000,
        cores=4,
        init="jitter+adapt_diag_grad",
        step=[pm.NUTS(target_accept=0.9)],
    )

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag_grad...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, α]


Sampling 4 chains for 2_000 tune and 10_000 draw iterations (8_000 + 40_000 draws total) took 24 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.


In [13]:
az.summary(trace, var_names=["~S", "~f", "~h"], kind="stats", hdi_prob=0.9)

,mean,sd,hdi_5%,hdi_95%
beta0,-6.533,0.649,-7.606,-5.478
beta1,0.257,0.235,-0.132,0.642
α,0.989,0.095,0.836,1.148
median0,516.337,91.421,366.680,657.844
median1,397.703,71.445,282.914,512.618


## Method 2: Imputed Censoring

This method is from [this notebook](https://docs.pymc.io/projects/examples/en/latest/survival_analysis/censored_data.html#censored-data-model1) by [Luis Mario Domenzain](https://github.com/domenzain), [George Ho](https://github.com/eigenfoo), and [Dr. Ben Vincent](https://github.com/drbenvincent). This method imputes the values using what is almost another likelihood (not sure if it actually meets the definition of one, so I'm calling the variable ```impute_censored```) in the model, with the right-censored values as lower bounds. Since the two likelihoods share the same priors, this ends up working nearly as well as the previous method.

In [ ]:
data = np.loadtxt("../data/gastric.txt")
x = data[:, 0].copy()
censored_vals = data[:, 1].copy()
y = data[:, 2].copy()

# we need to separate the observed values and the censored values
observed_mask = censored_vals == 0

censored = censored_vals[~observed_mask]
y_uncensored = y[observed_mask]
x_censored = x[~observed_mask]
x_uncensored = x[observed_mask]

In [ ]:
log2 = np.log(2)

with pm.Model() as m:
    beta0 = pm.Normal("beta0", 0, tau=0.0001)
    beta1 = pm.Normal("beta1", 0, tau=0.0001)
    α = pm.Exponential("α", 3)

    λ_censored = exp(beta0 + beta1 * x_censored)
    β_censored = λ_censored ** (-1 / α)

    λ_uncensored = exp(beta0 + beta1 * x_uncensored)
    β_uncensored = λ_uncensored ** (-1 / α)

    impute_censored = pm.Bound(
        "impute_censored",
        pm.Weibull.dist(alpha=α, beta=β_censored),
        lower=censored,
        shape=censored.shape[0],
    )

    likelihood = pm.Weibull(
        "likelihood",
        alpha=α,
        beta=β_uncensored,
        observed=y_uncensored,
        shape=y_uncensored.shape[0],
    )

    median0 = pm.Deterministic("median0", (log2 * exp(-beta0)) ** (1 / α))
    median1 = pm.Deterministic("median1", (log2 * exp(-beta0 - beta1)) ** (1 / α))

    trace = pm.sample(
        10000, tune=2000, cores=4, init="auto", step=[pm.NUTS(target_accept=0.95)]
    )

In [ ]:
az.summary(trace, hdi_prob=0.9, kind="stats")

In [ ]:
%watermark -n -u -v -iv -p aesara,aeppl